## 情感分布考察

In [55]:
import os
import pandas as pd
import numpy as np
import paddlehub as hub
import imageio

In [56]:
df = pd.read_csv("all_emo.csv")

In [57]:
df1 = df['content']

In [58]:
df = df[['content','评论类型']]

In [59]:
df

,content,评论类型
0,不行啊！这歌也太让人快乐了！洗澡的时候我直接扭成龙卷风！！！,热评
1,这派对也太危险了，男孩子在外面玩一定要好好保护自己[惊恐],热评
2,“右手托住你的心跳”怎么……托?,热评
3,你说你大姨妈走了让我晚上去你家宝守灵这活哥真干不了啊,热评
4,托住你的心跳这句可太高情商了,热评
...,...,...
128653,我在广东没看过雪,雪Distance
128654,第一,雪Distance
128655,我愿把胸膛解剖 公开脆弱的结构,雪Distance
128656,啊啊啊啊啊啊,雪Distance


In [60]:
# 加载模型
senta = hub.Module(name="senta_bilstm")

[2023-05-14 00:10:00,407] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object


In [62]:
test_text = df1.tolist()
for i in range(len(test_text)):
    test_text[i] = str(test_text[i])

In [63]:
results = senta.sentiment_classify(texts=test_text, use_gpu=False, batch_size=1)

In [64]:
results_df = pd.DataFrame(results)

In [65]:
df2 = pd.concat([df,results_df],axis=1)

In [66]:
df2

,content,评论类型,text,sentiment_label,sentiment_key,positive_probs,negative_probs
0,不行啊！这歌也太让人快乐了！洗澡的时候我直接扭成龙卷风！！！,热评,不行啊！这歌也太让人快乐了！洗澡的时候我直接扭成龙卷风！！！,0,negative,0.0493,0.9507
1,这派对也太危险了，男孩子在外面玩一定要好好保护自己[惊恐],热评,这派对也太危险了，男孩子在外面玩一定要好好保护自己[惊恐],0,negative,0.2686,0.7314
2,“右手托住你的心跳”怎么……托?,热评,“右手托住你的心跳”怎么……托?,0,negative,0.0207,0.9793
3,你说你大姨妈走了让我晚上去你家宝守灵这活哥真干不了啊,热评,你说你大姨妈走了让我晚上去你家宝守灵这活哥真干不了啊,0,negative,0.0189,0.9811
4,托住你的心跳这句可太高情商了,热评,托住你的心跳这句可太高情商了,0,negative,0.2604,0.7396
...,...,...,...,...,...,...,...
128653,我在广东没看过雪,雪Distance,我在广东没看过雪,1,positive,0.6763,0.3237
128654,第一,雪Distance,第一,1,positive,0.6791,0.3209
128655,我愿把胸膛解剖 公开脆弱的结构,雪Distance,我愿把胸膛解剖 公开脆弱的结构,0,negative,0.4541,0.5459
128656,啊啊啊啊啊啊,雪Distance,啊啊啊啊啊啊,1,positive,0.5955,0.4045


In [10]:
df2["new_sentiment_label"] = df2["positive_probs"].map(lambda x: 1 if x>=0.5 else 0)
df2["p"] = df2["positive_probs"].map(lambda x: 0.5 if x>=0 else 0)

In [67]:
#df2 = df2[['userid','timeStr','content','new_sentiment_label','p']]
df2 = df2[['content','positive_probs','评论类型']]

In [68]:
df2.head()

,content,positive_probs,评论类型
0,不行啊！这歌也太让人快乐了！洗澡的时候我直接扭成龙卷风！！！,0.0493,热评
1,这派对也太危险了，男孩子在外面玩一定要好好保护自己[惊恐],0.2686,热评
2,“右手托住你的心跳”怎么……托?,0.0207,热评
3,你说你大姨妈走了让我晚上去你家宝守灵这活哥真干不了啊,0.0189,热评
4,托住你的心跳这句可太高情商了,0.2604,热评


In [69]:
df2.to_csv("all_emotion_probs.csv",encoding='utf_8_sig')